<a><h1> Linear Regression with PyTorch


<h6>we're seeking to minimize the error between our model and the actual data, using a <em>loss function</em> like mean-squared-error.

<h6>we'll develop a collection of data points that appear random, but that fit a known linear equation y = 2x+1

<a><u><h3> Perform standard imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn


<a><u><h3> Create a column matrix of X values

In [8]:
X= torch.linspace(1,50,50).reshape(-1,1)
print(X)
print(X.shape)

#this will give 50 rows and 1 column

tensor([[ 1.],
        [ 2.],
        [ 3.],
        [ 4.],
        [ 5.],
        [ 6.],
        [ 7.],
        [ 8.],
        [ 9.],
        [10.],
        [11.],
        [12.],
        [13.],
        [14.],
        [15.],
        [16.],
        [17.],
        [18.],
        [19.],
        [20.],
        [21.],
        [22.],
        [23.],
        [24.],
        [25.],
        [26.],
        [27.],
        [28.],
        [29.],
        [30.],
        [31.],
        [32.],
        [33.],
        [34.],
        [35.],
        [36.],
        [37.],
        [38.],
        [39.],
        [40.],
        [41.],
        [42.],
        [43.],
        [44.],
        [45.],
        [46.],
        [47.],
        [48.],
        [49.],
        [50.]])
torch.Size([50, 1])


<a><u><h3> Create a "Random" array of error values

We want 50 random integer values that collectively cancel each other out.

In [ ]:
torch.manual_seed(71)
# this ensures whenever a random is generated it is same
# to obtain reproducible results

e=torch.randint(-8,9,(50,1),dtype=torch.float)

# (50,1) is the o/p Size
# -8,9 is the range

#to ensure that they cancel each other
print(e.sum())

<a><u><h3> C

<a><u><h3> 
 C

In [10]:
import random


In [16]:
random.seed(10)
print(random.random())

0.5714025946899135
